In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배 효과_naver_kin_933.txt'
    
f = open(openFileName, 'r', encoding='utf-8')
data = f.read()
f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가가': 0, '가감': 1, '가게': 2, '가격': 3, '가계': 4, '가계도': 5, '가공': 6, '가공품': 7, '가관': 8, '가구': 9, '가글': 10, '가금': 11, '가금류': 12, '가급': 13, '가기': 14, '가까이': 15, '가끔': 16, '가나': 17, '가난': 18, '가늠': 19, '가능': 20, '가능성': 21, '가다가': 22, '가닥': 23, '가도': 24, '가동': 25, '가득': 26, '가든': 27, '가라': 28, '가랑비': 29, '가래': 30, '가량': 31, '가레': 32, '가려움': 33, '가려움증': 34, '가렵': 35, '가렵거': 36, '가렵구': 37, '가렵기': 38, '가렵다': 39, '가렵습니': 40, '가령': 41, '가로': 42, '가로막': 43, '가로세로': 44, '가로수길': 45, '가루': 46, '가르침': 47, '가마': 48, '가맹': 49, '가면': 50, '가문': 51, '가물': 52, '가미': 53, '가바': 54, '가방': 55, '가변': 56, '가보': 57, '가부': 58, '가비': 59, '가빈': 60, '가빠오던': 61, '가쁜': 62, '가사': 63, '가산': 64, '가상': 65, '가설': 66, '가성': 67, '가성소다': 68, '가세': 69, '가소제': 70, '가속': 71, '가수': 72, '가쉽': 73, '가스': 74, '가스나': 75, '가스실': 76, '가슴': 77, '가슴앓이': 78, '가습기': 79, '가시': 80, '가신': 81, '가안': 82, '가압류': 83, '가액': 84, '가야': 85, '가야대': 86, '가약': 87, '가연성': 88, '가열': 89, '가오': 90, '가요': 91, '가용성': 92, '가운데': 93, '가운데손가락': 94, '가을': 95, '가의': 96

In [6]:
corpus_path = '전자담배 효과_naver_kin_933.txt'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(1393, 2), (1795, 1), (2054, 1), (2243, 1), (3677, 2), (4359, 1), (5121, 1), (5196, 1), (6914, 6), (7065, 1), (7228, 1), (7360, 1), (7636, 1), (8563, 4), (8830, 1), (9155, 5), (9273, 1), (9376, 1), (9831, 1), (10461, 3), (11669, 1), (12412, 1)]
[]
[]
[(629, 1), (1054, 3), (1214, 2), (1343, 5), (1559, 1), (2054, 4), (2093, 1), (2110, 1), (2243, 3), (2272, 1), (2433, 1), (2894, 1), (3187, 2), (3354, 1), (3743, 1), (4096, 1), (4439, 2), (4531, 1), (5003, 1), (5166, 1), (5329, 1), (5430, 1), (5638, 2), (6036, 1), (6649, 1), (6703, 1), (7028, 1), (7228, 1), (7472, 1), (7738, 1), (8305, 1), (8880, 1), (9026, 1), (9099, 1), (9152, 1), (9155, 2), (9281, 1), (9414, 1), (9457, 1), (10193, 1), (10338, 1), (10343, 1), (10461, 2), (11535, 1), (11577, 1), (11674, 2), (11699, 1), (11800, 2), (11905, 1), (11964, 1), (12246, 2), (12393, 1), (12412, 1)]
[]
[]
[(101, 1), (143, 1), (260, 1), (418, 1), (458, 1), (469, 1), (599, 6), (713, 1), (782, 1), (1422, 1), (2004, 1), (2093, 1), (2110, 1), (230

[]
[(101, 1), (957, 1), (1214, 1), (1924, 1), (2093, 1), (2096, 1), (2408, 1), (2433, 2), (2842, 1), (3414, 1), (5096, 1), (5520, 1), (6469, 1), (7078, 1), (7585, 1), (7723, 1), (8570, 1), (9155, 1), (9183, 2), (9959, 1), (11424, 3)]
[]
[]
[(138, 1), (599, 2), (1214, 1), (2054, 1), (2337, 1), (2842, 1), (4604, 1), (5599, 1), (6328, 1), (6684, 1), (7114, 2), (8969, 1), (9155, 1), (9424, 1), (9457, 1), (9959, 1), (10212, 2), (10461, 1), (10807, 1), (11546, 1), (11800, 1), (12029, 1)]
[]
[]
[(1343, 1), (1451, 1), (1506, 1), (2054, 2), (2243, 3), (4593, 1), (5696, 1), (6973, 1), (7416, 1), (7740, 1), (8080, 3), (8574, 2), (9152, 3), (9424, 2), (9710, 1), (9836, 1), (10436, 2), (10873, 1), (10967, 1), (11266, 1), (11430, 1), (12123, 1), (12253, 1), (12392, 2)]
[]
[]
[(101, 1), (260, 1), (578, 1), (1210, 1), (1226, 1), (1752, 1), (2054, 4), (2093, 1), (2243, 2), (2399, 1), (2565, 1), (2709, 1), (2748, 2), (3131, 1), (3331, 1), (4210, 1), (5532, 1), (6428, 1), (6643, 1), (6914, 1), (8726, 1),

[(101, 1), (418, 1), (558, 1), (599, 1), (871, 1), (1164, 1), (1210, 1), (1214, 1), (1795, 1), (2093, 2), (2110, 1), (2240, 1), (2243, 7), (2355, 1), (2738, 1), (2796, 1), (3239, 1), (3240, 1), (3331, 1), (4530, 2), (4611, 1), (4660, 1), (5307, 1), (6109, 1), (6643, 1), (6760, 1), (7289, 1), (8212, 1), (8396, 1), (8431, 1), (8552, 1), (8724, 2), (9042, 1), (9200, 1), (9750, 1), (10561, 2), (11577, 1), (11699, 1), (11800, 2), (11905, 1), (12412, 1)]
[]
[]
[(2, 1), (15, 1), (30, 1), (93, 2), (101, 2), (111, 1), (115, 5), (149, 2), (155, 1), (188, 1), (214, 2), (244, 1), (308, 1), (436, 2), (548, 1), (558, 1), (599, 2), (629, 1), (766, 2), (929, 1), (957, 1), (1214, 1), (1224, 1), (1321, 5), (1327, 1), (1343, 5), (1418, 1), (1497, 1), (1795, 1), (2054, 22), (2093, 4), (2110, 2), (2215, 1), (2243, 10), (2303, 1), (2329, 1), (2366, 1), (2398, 1), (2429, 8), (2669, 1), (2842, 1), (2894, 2), (3201, 1), (3312, 2), (3331, 1), (3387, 1), (3408, 3), (3411, 2), (3457, 2), (3597, 2), (3681, 1), (37

[]
[]
[(260, 1), (541, 1), (918, 1), (1857, 1), (2243, 1), (2272, 1), (2394, 1), (4575, 1), (6984, 1), (7240, 1), (7256, 1), (7842, 1), (8533, 1), (8726, 2), (9155, 1), (9235, 2), (9457, 1), (10000, 1), (11370, 1), (11577, 1), (11699, 3)]
[]
[]
[(2, 1), (15, 2), (30, 1), (93, 2), (101, 3), (111, 1), (115, 5), (149, 2), (155, 1), (188, 1), (214, 2), (244, 1), (308, 1), (436, 3), (548, 1), (558, 1), (599, 1), (629, 2), (697, 1), (766, 2), (929, 1), (957, 1), (1224, 1), (1321, 6), (1327, 1), (1343, 7), (1418, 1), (1497, 1), (1705, 1), (1795, 1), (1857, 1), (1907, 1), (2054, 27), (2093, 4), (2110, 2), (2215, 1), (2243, 11), (2272, 2), (2303, 1), (2329, 1), (2366, 1), (2398, 1), (2429, 7), (2669, 1), (2796, 1), (2842, 1), (2894, 3), (3201, 1), (3241, 1), (3266, 1), (3312, 2), (3316, 1), (3331, 1), (3387, 1), (3408, 3), (3411, 2), (3444, 1), (3457, 2), (3597, 3), (3681, 1), (3739, 1), (3877, 1), (3899, 1), (4062, 2), (4274, 1), (4477, 1), (4604, 1), (4611, 1), (4944, 1), (5120, 1), (5121, 1)

In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=12556, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 12556)
0.99999994


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\Users\BOGYUNG\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)